# Research Outputs Extraction

## 1. Project Setup

In [100]:
# Libraries
import json

### 1.1 Project Configurations

In [101]:
# Project Config
# proj_json_file = r"C:\Users\zooka\git_repos\learning\python\jinja_and_json\202008.03_001_project.json"
# proj_dir = r"C:\Users\zooka\git_repos\learning\python\jinja_and_json"

proj_json_file = r"C:\Users\he157349\source\repos\learning\python\jinja_and_json\202008.03_001_project.json"
proj_dir = r"C:\Users\he157349\source\repos\learning\python\jinja_and_json"

### 1.2 Load Configurations

In [102]:
# Opening JSON file
f = open(proj_json_file)

# Returns JSON object as a dictionary
proj_json = json.load(f)

# Closing file
f.close()

In [103]:
json.dumps(proj_json)

'{"project": {"id": "p2", "proj_no": "202008.03", "task_no": "001", "proj_type": "Linked Data Standard", "proj_title": "Ovarian Cancer", "princ_invest": "Prof Penelope Webb", "cohort_specs": [{"id": 1234, "name": "cohort_a", "description": "All individuals diagnosed with invasive ovarian cancer (including fallopian tube and primary peritoneal cancers) from 1/1/1984 to latest available (including children). ICD Codes (Principal diagnosis): C56 (ovarian cancer),C48.1, C48.2, C48.8 (primary peritoneal cancer),C57 (fallopian tube or other adnexal cancers)", "dat_id": "canc", "table_name_for_extraction": "DW_Research_Outputs.WACR.Vw_Cancer", "insert_into_table_name": "DW_Research_Outputs.PRJ.CohortSelect", "lnk_src_keys": ["TumourID"], "lnk_src": "CANC", "criteria": [{"field_name": "DiagnosisDate", "value": "19840101", "field_data_type": "date", "is_list": "FALSE", "condition_type": ">="}, {"field_name": "DiagnosisDate", "value": "20201231", "field_data_type": "date", "is_list": "FALSE", "c

#### 1.3 Call JSON to SQL function

In [116]:
import requests

#url_prod = "https://rosqlgenerator.azurewebsites.net/api/RoSqlGenerator"
url_dev = "http://localhost:7071/api/RoSqlGenerator"

# Convert json doc to string for Azure Function
proj_string = json.dumps(proj_json)

# Call Azure function
response = requests.request("GET", url_dev, data=proj_string)

# Check the response and show error if necessary
if not(response.status_code == 200):
    print('------- Error -------')
    print(response.text)
    print("Response code:{}".format(response.status_code))
    print("Error code description:{}".format(response.reason))
else:
    # Load all sql statements into a list of records (dict) one per sql statement
    proj_sql = json.loads(response.text)
    print('Success')

------- Error -------

Response code:500
Error code description:Internal Server Error


In [105]:
def find_sql(lst, key, value):
    for dictionary in lst:
        if dictionary.get(key) == value:
            return dictionary
    return None

## Project Header

In [106]:
# Load project details from json
project = proj_json['project']

print('------------------------')
print('Project ID : {id}'.format(**project))
print('Project No : {proj_no}'.format(**project))
print('Task No    : {task_no}'.format(**project))
print('Title      : {proj_title}'.format(**project))
print('PI         : {princ_invest}'.format(**project))
print('------------------------')

------------------------
Project ID : p2
Project No : 202008.03
Task No    : 001
Title      : Ovarian Cancer
PI         : Prof Penelope Webb
------------------------


## 2. Cohort Selection

### 2.1 Select Cohort Records

In [107]:
# Loop through all sql records and return cohort selection sql only
for rec_sql in proj_sql:
    if rec_sql.get('sql_type') == 'cohort_select':
        print(rec_sql['sql_txt'])



----------------------------------------------------
-- Cohort Selection
----------------------------------------------------
-- Proj ID : 
-- Proj No : 202008.03
-- Task No : 001
-- Title   : Ovarian Cancer
-- Principle Investigator: Prof Penelope Webb
----------------------------------------------------
INSERT INTO DW_Research_Outputs.PRJ.CohortSelect
       (
         CohId,
         LnkSrc,                 
         LnkSrcKey1     
       )

SELECT '1234'  as CohId,
       'CANC'  as LnkSrc,        TumourID as LnkSrcKey1    

  FROM DW_Research_Outputs.WACR.Vw_Cancer

 WHERE DiagnosisDate >= CAST('19840101' as date)
   AND DiagnosisDate <= CAST('20201231' as date)
   AND Behaviour = '3'
   AND (   CancerType like 'C56%'
        OR CancerType like 'C48.1%'
        OR CancerType like 'C48.2%'
        OR CancerType like 'C48.8%'
        OR CancerType like 'C57%'
       )
   AND (   TumourSiteCode like 'C481%'
        OR TumourSiteCode like 'C482%'
        OR TumourSiteCode like 'C48

### 2.2 Create Cohort

In [108]:
# Loop through all sql records and return cohort sql only
for rec_sql in proj_sql:
    if rec_sql.get('sql_type') == 'cohort':
        print(rec_sql['sql_txt'])



----------------------------------------------------
-- Cohort Listing
----------------------------------------------------
-- Proj ID : 
-- Proj No : 202008.03
-- Task No : 001
-- Title   : Ovarian Cancer
-- Principle Investigator: Prof Penelope Webb
----------------------------------------------------
INSERT INTO PRJ.Cohort
       (
         CohId,
         ChainId
       ) 

SELECT '1234'  as CohId,
       chain_id as ChainId

  FROM PRJ.CohortSelect coh
  JOIN DW_Transformed.LNK.LinkageKeys lnk 
    ON coh.LnkSrc     = lnk.src
   AND coh.LnkSrcKey1 = lnk.src_key_1

 WHERE coh.CohId = 1234;



----------------------------------------------------
-- Cohort Listing
----------------------------------------------------
-- Proj ID : 
-- Proj No : 202008.03
-- Task No : 001
-- Title   : Ovarian Cancer
-- Principle Investigator: Prof Penelope Webb
----------------------------------------------------
INSERT INTO PRJ.Cohort
       (
         CohId,
         ChainId
       ) 

SELECT '1235'

### 2.3 Cohort Checks

In [109]:
# Loop through all sql records and return cohort check sql only
for rec_sql in proj_sql:
    if rec_sql.get('sql_type') == 'cohort_checks':
        print(rec_sql['sql_txt'])



----------------------------------------------------
-- QA Checking : DiagnosisDate (frequency)
----------------------------------------------------
-- Proj ID : 
-- Proj No : 202008.03
-- Task No : 001
-- Title   : Ovarian Cancer
-- Principle Investigator: Prof Penelope Webb
----------------------------------------------------
INSERT INTO PRJ.CheckResults
       (
         CohId,
         LnkSrc,
         CheckType,
         CheckCol,
         CheckGrouping,
         CheckResultNum
       )

SELECT '1234'    as CohId,
       'CANC'    as Src,
       'frequency'    as CheckType,
       'DiagnosisDate'    as CheckCol,  
  
         year( DiagnosisDate ) as CheckGrouping,         count(*) as CheckResultNum


  FROM DW_Research_Outputs.WACR.Vw_Cancer dat
  JOIN PRJ.CohortSelect coh 
    ON 
       dat.TumourID = coh.LnkSrcKey1    

 WHERE coh.CohId      = '1234'
   AND coh.LnkSrc     = 'CANC' 
  
 GROUP BY 
  
       year( DiagnosisDate );



--------------------------------------------

## 3. Data Extraction

In [110]:
# Load project details from json
project = proj_json['project']

print(proj_json['project']['dataset_specs'])

[{'dat_id': 'bir', 'table_name_for_extraction': 'DW_Research_Outputs.DAT.vw_bir', 'lnk_src': 'BIR_BABY', 'lnk_src_keys': ['TumourID', 'RegID'], 'dat_date_min': '2003-01-01', 'dat_date_max': '2012-12-31', 'variables_requested': ['Presentation Date', 'Principal Diagnosis', 'Marital Status', 'Country of Birth'], 'db_fields_for_extraction': ['pres_rel', 'diag', 'mar', 'cob']}, {'dat_id': 'mor', 'table_name_for_extraction': 'DW_Research_Outputs.DAT.vw_mor_m', 'dat_date_min': '2003-01-01', 'dat_date_max': '2012-12-31', 'variables_requested': ['Presentation Date', 'Principal Diagnosis', 'Marital Status', 'Country of Birth'], 'db_fields_for_extraction': ['pres_rel', 'diag', 'mar', 'cob']}, {'dat_id': 'mor_r', 'table_name_for_extraction': 'DW_Research_Outputs.DAT.vw_mor_r', 'dat_date_min': '2003-01-01', 'dat_date_max': '2012-12-31', 'variables_requested': ['Presentation Date', 'Principal Diagnosis', 'Marital Status', 'Country of Birth'], 'db_fields_for_extraction': ['pres_rel', 'diag', 'mar', '

In [111]:
# Loop through all sql records and return extract data sql only
for rec_sql in proj_sql:
    if rec_sql.get('sql_type') == 'extraction':
        print(rec_sql['sql_txt'])

In [112]:
for rec_sql in proj_sql:
    print(rec_sql['sql_txt'])



----------------------------------------------------
-- Cohort Selection
----------------------------------------------------
-- Proj ID : 
-- Proj No : 202008.03
-- Task No : 001
-- Title   : Ovarian Cancer
-- Principle Investigator: Prof Penelope Webb
----------------------------------------------------
INSERT INTO DW_Research_Outputs.PRJ.CohortSelect
       (
         CohId,
         LnkSrc,                 
         LnkSrcKey1     
       )

SELECT '1234'  as CohId,
       'CANC'  as LnkSrc,        TumourID as LnkSrcKey1    

  FROM DW_Research_Outputs.WACR.Vw_Cancer

 WHERE DiagnosisDate >= CAST('19840101' as date)
   AND DiagnosisDate <= CAST('20201231' as date)
   AND Behaviour = '3'
   AND (   CancerType like 'C56%'
        OR CancerType like 'C48.1%'
        OR CancerType like 'C48.2%'
        OR CancerType like 'C48.8%'
        OR CancerType like 'C57%'
       )
   AND (   TumourSiteCode like 'C481%'
        OR TumourSiteCode like 'C482%'
        OR TumourSiteCode like 'C48